In [1]:
from IPython.display import Image, display
import PIL.Image
import io

import torch
import numpy as np
from processing_image import Preprocess
from modeling_frcnn import GeneralizedRCNN
from utils import Config
import utils


In [2]:
from datasets import load_dataset, Image
import pickle
import os
import cv2
import torch
import numpy as np
from torchvision import transforms

In [3]:
%load_ext autoreload
%autoreload 2

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [11]:
# load models and model components
frcnn_cfg = Config.from_pretrained("unc-nlp/frcnn-vg-finetuned")
frcnn_cfg.model.DEVICE = device

frcnn = GeneralizedRCNN.from_pretrained("unc-nlp/frcnn-vg-finetuned", config=frcnn_cfg).to(device)

image_preprocess = Preprocess(frcnn_cfg)

# bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
# visualbert_vqa = VisualBertForQuestionAnswering.from_pretrained("uclanlp/visualbert-vqa")

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at /nas/home/bchidamb/.cache/torch/transformers/57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [12]:
cifar10_data = load_dataset("cifar100")

Found cached dataset cifar100 (/nas/home/bchidamb/.cache/huggingface/datasets/cifar100/cifar100/1.0.0/f365c8b725c23e8f0f8d725c3641234d9331cd2f62919d1381d1baa5b3ba3142)


  0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
train_data = cifar10_data["train"]
test_data = cifar10_data["test"]
to_tensor = transforms.ToTensor()

In [14]:
for idx, img_data in enumerate(train_data):
    print(img_data)
    if idx % 1000 == 0:
        print("On idx " + str(idx))
    image = np.array(img_data["img"])
    # run frcnn
    images, sizes, scales_yx = image_preprocess(image)
    output_dict = frcnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=frcnn_cfg.max_detections,
        return_tensors="pt",
    )
    features = output_dict.get("roi_features")
    
#     visual_embeddings.append(features)
#     labels.append(img_data["label"])
    break
output_dict

{'img': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7FBE8CEBAA70>, 'fine_label': 19, 'coarse_label': 11}
On idx 0


OrderedDict([('obj_ids',
              tensor([[ 364,  364,  364,  540,  540,  364,  364,  540,  364,  540,  540,  364,
                        540,  540,  364,  540,  540,  364,  364,  540,  540,  540,  540,  956,
                        956,  956,  540,  540,  956,  540,  956,  956,  956, 1021,  540,  956]])),
             ('obj_probs',
              tensor([[0.7614, 0.6832, 0.6828, 0.6101, 0.5694, 0.5662, 0.5573, 0.5517, 0.5334,
                       0.5088, 0.5017, 0.4458, 0.4230, 0.3948, 0.3843, 0.3834, 0.3656, 0.3483,
                       0.3362, 0.3353, 0.2942, 0.2785, 0.2744, 0.2637, 0.2476, 0.2122, 0.2046,
                       0.1994, 0.1621, 0.1527, 0.1514, 0.1427, 0.1417, 0.1307, 0.1284, 0.1181]])),
             ('attr_ids',
              tensor([[210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210,
                       210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210, 210,
                       210, 210, 210, 210, 210, 210, 210, 210]]

In [15]:
output_dict['normalized_boxes'].shape

torch.Size([1, 36, 4])

In [17]:
print("Generating " + str(len(train_data)) + " embeddings")

visual_embeddings = []
labels = []
bounding_boxes = []
fine_labels = []
coarse_labels = []

for idx, img_data in enumerate(train_data):
    if idx % 1000 == 0:
        print("On idx " + str(idx))
    image = np.array(img_data["img"])
    # run frcnn
    images, sizes, scales_yx = image_preprocess(image)
    output_dict = frcnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=frcnn_cfg.max_detections,
        return_tensors="pt",
    )
    features = output_dict.get("roi_features")
    box = output_dict.get("normalized_boxes")
    
    visual_embeddings.append(features)
    bounding_boxes.append(box)
    fine_labels.append(img_data["fine_label"])
    coarse_labels.append(img_data["coarse_label"])


Generating 50000 embeddings
On idx 0
On idx 1000
On idx 2000
On idx 3000
On idx 4000
On idx 5000
On idx 6000
On idx 7000
On idx 8000
On idx 9000
On idx 10000
On idx 11000
On idx 12000
On idx 13000
On idx 14000
On idx 15000
On idx 16000
On idx 17000
On idx 18000
On idx 19000
On idx 20000
On idx 21000
On idx 22000
On idx 23000
On idx 24000
On idx 25000
On idx 26000
On idx 27000
On idx 28000
On idx 29000
On idx 30000
On idx 31000
On idx 32000
On idx 33000
On idx 34000
On idx 35000
On idx 36000
On idx 37000
On idx 38000
On idx 39000
On idx 40000
On idx 41000
On idx 42000
On idx 43000
On idx 44000
On idx 45000
On idx 46000
On idx 47000
On idx 48000
On idx 49000


In [18]:
cifar10_train_embeddings = {
    "embeddings": visual_embeddings,
    "bounding_box": bounding_boxes,
    "labels": labels
}

with open("cifar100-train-embeddings.pkl", "wb") as f:
    pickle.dump(cifar10_train_embeddings, f)

In [19]:
print("Generating " + str(len(test_data)) + " embeddings")

visual_embeddings = []
labels = []
bounding_boxes = []
fine_labels = []
coarse_labels = []

for idx, img_data in enumerate(test_data):
    if idx % 1000 == 0:
        print("On idx " + str(idx))
    image = np.array(img_data["img"])
    # run frcnn
    images, sizes, scales_yx = image_preprocess(image)
    output_dict = frcnn(
        images,
        sizes,
        scales_yx=scales_yx,
        padding="max_detections",
        max_detections=frcnn_cfg.max_detections,
        return_tensors="pt",
    )
    features = output_dict.get("roi_features")
    box = output_dict.get("normalized_boxes")
    
    visual_embeddings.append(features)
    bounding_boxes.append(box)
    fine_labels.append(img_data["fine_label"])
    coarse_labels.append(img_data["coarse_label"])


Generating 10000 embeddings
On idx 0
On idx 1000
On idx 2000
On idx 3000
On idx 4000
On idx 5000
On idx 6000
On idx 7000
On idx 8000
On idx 9000


In [20]:
cifar10_test_embeddings = {
    "embeddings": visual_embeddings,
    "bounding_box": bounding_boxes,
    "labels": labels
}

with open("cifar100-test-embeddings.pkl", "wb") as f:
    pickle.dump(cifar10_test_embeddings, f)